In [ ]:
import sempy.fabric as fabric
from sempy.fabric.exceptions import FabricHTTPException

# Listar capacidades de Fabric
Debemos escoger la capacidad que vamos a utilizar para asignarla al o las áreas de trabajo que se crearán.

In [ ]:
fabric.list_capacities()

In [ ]:
projectName:str = "DEV_datadaX"
layers:list[str] = ["01_BRONZE","02_SILVER","03_GOLD"]
capacityId:str = "399c717e-8ef7-4d43-a7f6-61ff7fc07e71"

# Definición de funciones

In [ ]:
def create_workspace(workspace_name:str,capacity_id:str,description:str):
    """
    Kilian Baccaro | datagym.es

    Función para crear un área de trabajo.

    Parámetros:
    - workspace_name (str): Nombre del área de trabajo a crear.
    - capacity_id (str): ID de la capacidad en la que se asignará el área de trabajo.
    - description (str): Descripción del área de trabajo.

    Devuelve:
    - str: ID del área de trabajo creada si la operación es exitosa.
    - None: En caso de error.
    
    """

    try:
        workspaceId = fabric.create_workspace(workspace_name, capacity_id, description)
        print(f"✅ Área de trabajo {workspace_name} creada con éxito:")
        print(f"ℹ️ Workspace ID: {workspaceId}")
        return workspaceId

    except Exception as error:
        print("❌ Error al crear el área de trabajo: ", error)
        return None


def search_workspace(workspace_name: str):
    """
    Kilian Baccaro | datagym.es

    Busca un área de trabajo en Microsoft Fabric por su nombre.

    Parámetros:
    - workspace_name (str): Nombre del área de trabajo a buscar.

    Devuelve:
    - pd.DataFrame: Un DataFrame con la información del área de trabajo si existe.
    - None: Si ocurre un error o no se encuentra el área de trabajo.

    """

    try:
        workspace = fabric.list_workspaces(filter=f"name eq '{workspace_name}'")
        return workspace
    except Exception as error:
        print("❌ Error al buscar el área de trabajo: ", error)
        return None

def list_lakehouses(workspace_id:str):
    """
    Kilian Baccaro | datagym.es

    Lista todos los lakehouses disponibles en un área de trabajo en Microsoft Fabric.

    Esta función utiliza la API de Microsoft Fabric para obtener una lista de todos los lakehouses
    asociados con un área de trabajo específica, identificada por el parámetro `workspace_id`. 
    Si la operación es exitosa, devuelve una lista con los lakehouses. En caso de que ocurra un 
    error durante la operación, captura la excepción y muestra un mensaje de error.

    Parámetros:
    - workspace_id (str): El identificador del área de trabajo en Microsoft Fabric para la cual 
      se desean obtener los lakehouses.

    Devuelve:
    - list: Una lista de los lakehouses disponibles en el área de trabajo. Si ocurre un error, 
      devuelve `None`.

    """

    try:
        lakehouses = notebookutils.lakehouse.list(workspace_id)
        return lakehouses
    except Exception as error:
        print("❌ Error al listar los lakehouses: ", error)
        return None


def create_lakehouse(lakehouse_name: str, workspace_id: str):
    """
    Kilian Baccaro | datagym.es
    
    Crea un nuevo lakehouse en un área de trabajo de Microsoft Fabric.

    Esta función interactúa con la API de Microsoft Fabric para crear un nuevo lakehouse
    dentro de un área de trabajo específica, identificada por el parámetro `workspace_id`. 
    El lakehouse se crea con el nombre especificado por el parámetro `lakehouse_name`, 
    y se pueden personalizar otros parámetros como la descripción y el número máximo de intentos 
    para la creación. Si la operación es exitosa, se devuelve el id del lakehouse creado. 
    En caso de error, la función captura la excepción y muestra un mensaje de error en consola.

    Parámetros:
    - lakehouse_name (str): El nombre que se asignará al nuevo lakehouse.
    - workspace_id (str): El identificador del área de trabajo en Microsoft Fabric donde se creará 
      el lakehouse.

    Devuelve:
    - str: El ID del lakehouse creado en Microsoft Fabric. Si ocurre un error durante la 
      creación, devuelve `None`.

    """

    description = ""
    max_attempts = 10

    try:
        lakehouseId = fabric.create_lakehouse(lakehouse_name, description, max_attempts, workspace_id)
        return lakehouseId
    except Exception as error:
        print(f"❌ Error al crear el lakehouse {lakehouse_name}: ", error)
        return None

# Ejecución

In [ ]:
%%time

"""
ÁREA DE TRABAJO
   
Se comprueba si existe el área de trabajo. Si existe, obtenemos el workspaceId, sino, creamos el área de trabajo
y obtenemos el workspaceId.
   
"""

print("ℹ️ El script está configurado para crear un área de trabajo para cada capa")
print("ℹ️ Inicializando la creación del área de trabajo...")

workspaces_df = fabric.list_workspaces()
workspacesExistentes = workspaces_df[["Name", "Id"]].to_dict(orient="records")

for layer in layers:
    workspace_name = f"{projectName}_{layer}"
    workspace = search_workspace(workspace_name)

    if workspace is not None and not workspace.empty:
        print(f"ℹ️ El workspace {workspace_name} ya existe. Se crearán los objetos sobre esta área de trabajo.")
        print(f"ℹ️ Workspace ID: {workspace.iloc[0]['Id']}")
        print(f"ℹ️ Capacity ID: {workspace.iloc[0]['Capacity Id']}")
        workspace_id = workspace.iloc[0]['Id']
    else:
        print(f"ℹ️ Creando área de trabajo {workspace_name}...")
        workspace_id = create_workspace(workspace_name, capacityId, None)
    
    if workspace_id is None:
        notebookutils.session.stop()

    print("")

    """
    LAKEHOUSE
    
    Se crea el lakehouse correspondiente de la capa
    
    """

    print("ℹ️ Inicializando la creación de los lakehouses...")
    lakehousesList = list_lakehouses(workspace_id)

    lakehousesExistentes = [
        {"name": item["displayName"], "id": item["id"]}
        for item in lakehousesList
    ]

    lakehouseName = f"{projectName}_{layer}_lh"

    if lakehouseName not in [lh["name"] for lh in lakehousesExistentes]:
        print(f"ℹ️ Creando lakehouse {lakehouseName}...")
        lakehouseId = create_lakehouse(lakehouseName, workspace_id)
        if lakehouseId is not None:
            print(f"✅ Lakehouse creado con éxito: {lakehouseId}")
    
    else:
        print(f"ℹ️ El lakehouse {lakehouseName} ya existe.")

    print("")


print(f"✅ Ejecución finalizada.")